In [1]:
!pip install geopandas --user -q

In [2]:
import pyproj
import geopyspark as gps
import fiona
import geopandas as gpd

from functools import partial
from pyspark import SparkContext
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, Polygon, shape
from shapely.ops import transform

from overlay.data import data, mask

In [3]:
conf = gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '12G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

# Functions

In [20]:
def mapp(tiled):
    pyramid_layer = tiled.pyramid()
    tms = gps.TMS.build(pyramid_layer, gps.ColorMap.build(pyramid_layer.get_histogram(), 'magma'))
    for l in M.layers:
        M.remove_layer(l)
    M.add_layer(TMSRasterData(tms), name="layer")

# Study Area

In [87]:
base_gpd = gpd.read_file("data/read/boundaries/houston.shp")
base_poly = [area_gpd.geometry[0]]

In [88]:
base_raster = gps.rasterize(houston_area_geoms,
                                        4326,
                                        12,
                                        1,
                                        gps.CellType.INT8)

In [89]:
base_tiled = base_raster.tile_to_layout(gps.GlobalLayout(), 3857)

#### Visualize Study Area

In [90]:
mapp(base_tiled)

# Storm Surge

In [94]:
storm_raster = gps.geotiff.get(gps.LayerType.SPATIAL,
                    '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif',
                     max_tile_size=128,
                     num_partitions=32)

In [95]:
rmap = {}
for x in range(2, 24, 2):
    rmap[x + 1] = int(x / 2)

storm_reclass = surge_tiled.reclassify(value_map=rmap,
                                    data_type=int,
                                    replace_nodata_with=1,
                                    classification_strategy=gps.ClassificationStrategy.LESS_THAN_OR_EQUAL_TO)

In [99]:
storm_tiled = storm_reclass.tile_to_layout(base_tiled.layer_metadata)

#### Visualize Storm Surge

In [97]:
mapp(storm_tiled)

#### Add Storm Surge and Houston Area

In [102]:
storm_overlay = storm_tiled * base_tiled 

In [103]:
mapp(storm_overlay)

# Sea Level Rise

In [104]:
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [105]:
sea_poly = []
with fiona.open(gdb, layer='TX_HGX_slr_5ft') as source:
    slr_crs = source.crs['init']
    for p in source:
        sea_poly += MultiPolygon(shape(p['geometry']))

In [106]:
sea_raster = gps.rasterize(slr,
                           4326,
                           12,
                           5,
                           gps.CellType.INT8)

In [107]:
sea_reclass = sea_raster.reclassify(value_map={5: 5},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

In [108]:
sea_tiled = sea_reclass.tile_to_layout(gps.GlobalLayout(), 3857)

#### Union And Aggregate Sea Level and Houston

In [109]:
sea_unioned_base = gps.geotrellis.union(layers=[base_tiled, sea_tiled])

In [142]:
sea_aggregated_base = sea_unioned_base.aggregate_by_cell(operation=gps.geotrellis.constants.Operation.MAX)

In [143]:
sea_overlay = sea_aggregated_base.reclassify(value_map={5: 5, 1:1},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

#### Visualizing Sea Level Rise

In [112]:
mapp(sea_overlay)

# Flood Hazard

In [113]:
flood_poly = data.read_from_shp('/home/hadoop/notebooks/data/write/overlay-layers/flood/Fld_Haz_ar_wm_houston.shp')

In [114]:
flood_raster = gps.rasterize(geoms=flood_poly,
                                 crs = 3857,
                                 zoom=12,
                                 fill_value = 5,
                                 options = gps.RasterizerOptions(
                                     includePartial=True, 
                                     sampleType='PixelIsArea'),
                                 cell_type = gps.CellType.INT8,
                                 num_partitions = 50)

In [144]:
flood_reclass = flood_raster.reclassify(value_map={5: 5},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

In [145]:
flood_tiled = flood_reclass.tile_to_layout(gps.GlobalLayout(), 3857)

#### Union And Aggregate Flood Hazard and Houston

In [146]:
flood_unioned_base = gps.geotrellis.union(layers=[flood_tiled, base_tiled])

In [147]:
flood_aggregated_base = flood_unioned_base.aggregate_by_cell(operation=gps.geotrellis.constants.Operation.MAX)

In [148]:
flood_overlay = flood_aggregated_base.reclassify(value_map={5: 5, 1:1},
                                   data_type=int,
                                   replace_nodata_with=1,
                                   classification_strategy=gps.ClassificationStrategy.EXACT)

#### Visualizing Sea Level Rise

In [137]:
mapp(flood_overlay)

# Overlay

In [149]:
overlay = flood_overlay + sea_overlay + storm_overlay - 3

In [150]:
mapp(overlay)